In [240]:
# !pip install plotly
# !pip install pythainlp
# !pip install stop_words
# !pip install nltk

### Import Library

In [241]:
# import libraries
import pandas as pd
import plotly.express as px
import re
import pickle
import nltk
nltk.download('words')
nltk.download('omw-1.4')

[nltk_data] Downloading package words to C:\Users\FACT-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\FACT-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

### Import CSV

In [242]:
# import glob, os

# all_files = glob.glob("../../data_lyrics/thaisongs/*.csv")

# li = []

# for filename in all_files:
#     song_df = pd.read_csv(filename, index_col=None, header=0)
#     li.append(song_df)

# song_df = pd.concat(li, axis=0, ignore_index=True)

In [243]:
song_df = pd.read_csv('../../data_lyrics/35_song_per_artist.csv')
song_df.dtypes

artist       object
song_name    object
href         object
lyric        object
lines         int64
dtype: object

## Data Preparation

### Clean Data

In [244]:
# remove !@#$%^&*()_+
lyric_in_round_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\((.*?)\)',s))), [])
lyric_in_square_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\[(.*?)\]',s))), [])
lyric_in_curly_brackets = sum(list(song_df['lyric'].map(lambda s: re.findall(r'\{(.*?)\}',s))), [])
dot = sum(list(song_df['lyric'].map(lambda s: re.findall('\.',s))), [])
star = sum(list(song_df['lyric'].map(lambda s: re.findall('\*',s))), [])
semi_colon = sum(list(song_df['lyric'].map(lambda s: re.findall('\;',s))), [])
colon = sum(list(song_df['lyric'].map(lambda s: re.findall('\:',s))), [])
exclam_mark = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\!",s))), [])
slash = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\/",s))), [])
backslash = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\\",s))), [])
question_mark = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\?",s))), [])
hashtag = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\#",s))), [])
percent = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\%",s))), [])
plus = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\+",s))), [])
minus = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\-",s))), [])
comma = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\,",s))), [])
sg_quote = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\'",s))), [])
db_quote = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\"",s))), [])
dollar_sign = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\$",s))), [])
ampersand = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\&",s))), [])
underscore = sum(list(song_df['lyric'].map(lambda s: re.findall(r"\_",s))), [])


song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\((.*?)\)', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\[(.*?)\]', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\{(.*?)\}', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\.', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\*', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\;', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\:', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r"\!", '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r"\/", '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\\', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r"\?", '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r"\#", '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r"\%", '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r"\+", '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r"\-", '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r"\,", '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r"\'", '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\"', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\$', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\&', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\(|\)', '', s))
song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r'\_', '', s))


print('Number of round brackets: {}'.format(len(lyric_in_round_brackets)))
print('Number of square brackets: {}'.format(len(lyric_in_square_brackets)))
print('Number of curly brackets: {}'.format(len(lyric_in_curly_brackets)))
print('Number of dot: {}'.format(len(dot)))
print('Number of star: {}'.format(len(star)))
print('Number of semi colon: {}'.format(len(semi_colon)))
print('Number of colon: {}'.format(len(colon)))
print('Number of exclamation mark: {}'.format(len(exclam_mark)))
print('Number of slash: {}'.format(len(slash)))
print('Number of backslash: {}'.format(len(slashR)))
print('Number of question mark: {}'.format(len(question_mark)))
print('Number of hashtag: {}'.format(len(hashtag)))
print('Number of percent: {}'.format(len(percent)))
print('Number of plus: {}'.format(len(plus)))
print('Number of minus: {}'.format(len(minus)))
print('Number of comma: {}'.format(len(comma)))
print('Number of single quote: {}'.format(len(sg_quote)))
print('Number of double quote: {}'.format(len(db_quote)))
print('Number of dollar sign: {}'.format(len(dollar_sign)))
print('Number of ampersand: {}'.format(len(ampersand)))
print('Number of underscore: {}'.format(len(underscore)))

Number of round brackets: 0
Number of square brackets: 0
Number of curly brackets: 0
Number of dot: 0
Number of star: 0
Number of semi colon: 0
Number of colon: 0
Number of exclamation mark: 0
Number of slash: 0
Number of backslash: 0
Number of question mark: 0
Number of hashtag: 0
Number of percent: 0
Number of plus: 0
Number of minus: 0
Number of comma: 0
Number of single quote: 0
Number of double quote: 0
Number of dollar sign: 0
Number of ampersand: 0
Number of underscore: 0


In [245]:
song_df

,artist,song_name,href,lyric,lines
0,bird_thongchai,Okay,/music/thailyric/13588,ไม่ว่าจะเป็นยังไง Baby its Okay\rไม่ว่าจะเกิดอ...,57
1,bird_thongchai,กว่าจักรวาล,/music/thailyric/13978,จะยอมนั่งจรวด ไปตรวจดาวอังคาร\rหากว่าที่แห่งนั...,31
2,bird_thongchai,กำแพง,/music/thailyric/14111,ถ้าเคยพบเจอ กำแพงที่ดูทั้งใหญ่และสูงชัน\rเธอรู...,39
3,bird_thongchai,ชีวิตเดี่ยว,/music/thailyric/13796,อยู่ตรงนี้แค่เพียงลำพัง\rกับความเหงาที่เป็นดั่...,50
4,bird_thongchai,ผู้ต้องหา,/music/thailyric/13825,แค่ตัวคนเดียวไม่ตายล่ะมั้ง\rถามใจกี่ครั้งก็ยัง...,65
...,...,...,...,...,...
2235,gavin_d,พ่อตาครับ,/music/thailyric/17862,พ่อตาผมโคตรโหด ผมไม่เคยเจอใครที่หนวดเยอะขนาดนี...,48
2236,gavin_d,ไม่หวังให้ตอบ (No Reply),/music/thailyric/13101,ฉันไม่ได้ตั้งใจที่จะส่งไปหาเธอ\rหน้าจอมันเบลอเ...,60
2237,gavin_d,ยิ่งลืมยิ่งคิดถึง,/music/thailyric/20476,ทุกครั้งที่ฉันนะคิดจะลืม\rมันคือเวลาที่คิดถึงม...,58
2238,gavin_d,รัก?,/music/thailyric/14276,ไม่เป็นไรใช่ไหม\rนี่คือคำถาม หรือความห่วงใย\rถ...,98


In [246]:
# count number of lines
song_df['lines'] = song_df['lyric'].map(lambda t: len(re.findall(r'\r', t)))

In [247]:
# # remove line breaks
# song_df['lyric'] = song_df['lyric'].map(lambda s: re.sub(r' \r|\r', '', s))

### Tokenise

In [248]:
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus.common import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
# from pythainlp.ulmfit import process_thai

In [249]:
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))
p_stemmer = PorterStemmer()

In [250]:
def split_word(text):

    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และภาษาอังกฤษ
    #tokens = [i for i in tokens if not i in th_stop and not i in en_stop]
    
    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    # tokens_temp=[]
    # for i in tokens:
    #     w_syn = wordnet.synsets(i)
    #     if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
    #         tokens_temp.append(w_syn[0].lemma_names('tha')[0])
    #     else:
    #         tokens_temp.append(i)
    
    # tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

In [251]:
song_df['words'] = song_df['lyric'].map(lambda s: re.sub(r' \r|\r', '', s)).map(lambda s: split_word(s))

# This function may take longer than 2 minutes.

In [252]:
song_df[['lyric','words']].tail()

,lyric,words
2235,พ่อตาผมโคตรโหด ผมไม่เคยเจอใครที่หนวดเยอะขนาดนี...,"[พ่อตา, ผม, โคตร, โหด, ผม, ไม่, เคย, เจอ, ใคร,..."
2236,ฉันไม่ได้ตั้งใจที่จะส่งไปหาเธอ\rหน้าจอมันเบลอเ...,"[ฉัน, ไม่, ได้, ตั้งใจ, ที่จะ, ส่ง, ไปหา, เธอ,..."
2237,ทุกครั้งที่ฉันนะคิดจะลืม\rมันคือเวลาที่คิดถึงม...,"[ทุกครั้งที่, ฉัน, นะ, คิด, จะ, ลืม, มัน, คือ,..."
2238,ไม่เป็นไรใช่ไหม\rนี่คือคำถาม หรือความห่วงใย\rถ...,"[ไม่เป็นไร, ใช่ไหม, นี่, คือ, คำถาม, หรือ, ควา..."
2239,ไม่เคยมีใครน่ารักเท่ากับเธอ\rเจอกี่ครั้งก็ยังล...,"[ไม่, เคย, มี, ใคร, น่ารัก, เท่ากับ, เธอ, เจอ,..."


In [253]:
song_df['words']

0       [ไม่, ว่า, จะ, เป็น, ยังไง, babi, it, okay, ไม...
1       [จะ, ยอม, นั่ง, จรวด, ไป, ตรวจ, ดาวอังคาร, หาก...
2       [ถ้า, เคย, พบ, เจอ, กำแพง, ที่, ดู, ทั้ง, ใหญ่...
3       [อยู่, ตรงนี้, แค่, เพียงลำพัง, กับ, ความเหงา,...
4       [แค่, ตัว, คนเดียว, ไม่, ตาย, ล่ะ, มั้ง, ถาม, ...
                              ...                        
2235    [พ่อตา, ผม, โคตร, โหด, ผม, ไม่, เคย, เจอ, ใคร,...
2236    [ฉัน, ไม่, ได้, ตั้งใจ, ที่จะ, ส่ง, ไปหา, เธอ,...
2237    [ทุกครั้งที่, ฉัน, นะ, คิด, จะ, ลืม, มัน, คือ,...
2238    [ไม่เป็นไร, ใช่ไหม, นี่, คือ, คำถาม, หรือ, ควา...
2239    [ไม่, เคย, มี, ใคร, น่ารัก, เท่ากับ, เธอ, เจอ,...
Name: words, Length: 2240, dtype: object

In [254]:
song_df['n_words'] = song_df['words'].map(len)

### EDA

In [255]:
# number of songs
print('number of songs: ', str(len(song_df)))

# number of artists
print('number of artists: ', str(len(song_df['artist'].unique())))

number of songs:  2240
number of artists:  64


In [256]:
# distribution songs per artist
song_count_df = song_df.groupby('artist')[['song_name']].count()
print()

fig = px.histogram(song_count_df, x='song_name', title='Songs per artist', labels={'song_name': 'Songs'})
fig.show()

In [257]:
# distribution words per song
fig = px.histogram(song_df, x='n_words', title='Words per song')
fig.show()

Beware! This block can be run only once.

In [258]:
# # create dataframe with lists of artists
song_df['words_str'] = song_df['words'].map(lambda lst: ' '.join(lst))

# map text to artists
words_to_artist = {}
for tp in song_df[['artist', 'words_str']].itertuples(index=False):
    artist = tp[0]
    words = tp[1]
    if words in words_to_artist:
        words_to_artist[words].append(artist)
    else:
        words_to_artist[words] = [artist]

# insert list of artists to dataframe
song_df['artists'] = song_df['words_str'].map(words_to_artist)
song_df['duplicates'] = song_df['artists'].map(len) - 1

# convert list of artists to set of artists
song_df['artists'] = song_df['artists'].map(set)
song_df['n_artists'] = song_df['artists'].map(len)

# remove duplicate songs
artist_text_df = song_df.drop_duplicates('words_str')

In [259]:
song_df

,artist,song_name,href,lyric,lines,words,n_words,words_str,artists,duplicates,n_artists
0,bird_thongchai,Okay,/music/thailyric/13588,ไม่ว่าจะเป็นยังไง Baby its Okay\rไม่ว่าจะเกิดอ...,57,"[ไม่, ว่า, จะ, เป็น, ยังไง, babi, it, okay, ไม...",343,ไม่ ว่า จะ เป็น ยังไง babi it okay ไม่ ว่า จะ ...,{bird_thongchai},0,1
1,bird_thongchai,กว่าจักรวาล,/music/thailyric/13978,จะยอมนั่งจรวด ไปตรวจดาวอังคาร\rหากว่าที่แห่งนั...,31,"[จะ, ยอม, นั่ง, จรวด, ไป, ตรวจ, ดาวอังคาร, หาก...",239,จะ ยอม นั่ง จรวด ไป ตรวจ ดาวอังคาร หาก ว่าที่ ...,{bird_thongchai},0,1
2,bird_thongchai,กำแพง,/music/thailyric/14111,ถ้าเคยพบเจอ กำแพงที่ดูทั้งใหญ่และสูงชัน\rเธอรู...,39,"[ถ้า, เคย, พบ, เจอ, กำแพง, ที่, ดู, ทั้ง, ใหญ่...",316,ถ้า เคย พบ เจอ กำแพง ที่ ดู ทั้ง ใหญ่ และ สูงช...,{bird_thongchai},0,1
3,bird_thongchai,ชีวิตเดี่ยว,/music/thailyric/13796,อยู่ตรงนี้แค่เพียงลำพัง\rกับความเหงาที่เป็นดั่...,50,"[อยู่, ตรงนี้, แค่, เพียงลำพัง, กับ, ความเหงา,...",250,อยู่ ตรงนี้ แค่ เพียงลำพัง กับ ความเหงา ที่ เป...,{bird_thongchai},0,1
4,bird_thongchai,ผู้ต้องหา,/music/thailyric/13825,แค่ตัวคนเดียวไม่ตายล่ะมั้ง\rถามใจกี่ครั้งก็ยัง...,65,"[แค่, ตัว, คนเดียว, ไม่, ตาย, ล่ะ, มั้ง, ถาม, ...",416,แค่ ตัว คนเดียว ไม่ ตาย ล่ะ มั้ง ถาม ใจ กี่ คร...,{bird_thongchai},0,1
...,...,...,...,...,...,...,...,...,...,...,...
2235,gavin_d,พ่อตาครับ,/music/thailyric/17862,พ่อตาผมโคตรโหด ผมไม่เคยเจอใครที่หนวดเยอะขนาดนี...,48,"[พ่อตา, ผม, โคตร, โหด, ผม, ไม่, เคย, เจอ, ใคร,...",434,พ่อตา ผม โคตร โหด ผม ไม่ เคย เจอ ใคร ที่ หนวด ...,{gavin_d},0,1
2236,gavin_d,ไม่หวังให้ตอบ (No Reply),/music/thailyric/13101,ฉันไม่ได้ตั้งใจที่จะส่งไปหาเธอ\rหน้าจอมันเบลอเ...,60,"[ฉัน, ไม่, ได้, ตั้งใจ, ที่จะ, ส่ง, ไปหา, เธอ,...",381,ฉัน ไม่ ได้ ตั้งใจ ที่จะ ส่ง ไปหา เธอ หน้าจอ ม...,{gavin_d},0,1
2237,gavin_d,ยิ่งลืมยิ่งคิดถึง,/music/thailyric/20476,ทุกครั้งที่ฉันนะคิดจะลืม\rมันคือเวลาที่คิดถึงม...,58,"[ทุกครั้งที่, ฉัน, นะ, คิด, จะ, ลืม, มัน, คือ,...",317,ทุกครั้งที่ ฉัน นะ คิด จะ ลืม มัน คือ เวลา ที่...,{gavin_d},0,1
2238,gavin_d,รัก?,/music/thailyric/14276,ไม่เป็นไรใช่ไหม\rนี่คือคำถาม หรือความห่วงใย\rถ...,98,"[ไม่เป็นไร, ใช่ไหม, นี่, คือ, คำถาม, หรือ, ควา...",511,ไม่เป็นไร ใช่ไหม นี่ คือ คำถาม หรือ ความห่วงใย...,{gavin_d},0,1


## Export the clean data

In [260]:
song_df.to_pickle('../../data_lyrics/pd/clean_th_songs_35.pkl')